## Arkouda Registration Example
In this example we will:
- Create random pdarrays, Strings, Categoricals, SegArrays, and GroupBys
- Register these objects
- Verify their registration status using `ak.list_registry`, `.is_registered`, and `.info`
- Convert `ak.information`'s JSON return string into a python object 
- Remove all non-registered objects from the symbol table using `ak.clear`
- Disconnect from the arkouda server
- Reconnect to the arkouda server
- Attach to all registered objects
- Unregister everything
- Shutdown the server

Arkouda functions used:
- `ak.connect`
- `ak.randint`
- `ak.random_strings_uniform`
- `ak.pdarray.register`
- `ak.Strings.register`
- `ak.Categorical.register`
- `ak.SegArray.register`
- `ak.GroupBy.register`
- `ak.list_registry`
- `ak.pdarray.is_registered`
- `ak.Strings.is_registered`
- `ak.Categorical.is_registered`
- `ak.SegArray.is_registered`
- `ak.GroupBy.is_registered`
- `ak.Strings.info`
- `ak.Strings.pretty_print_info`
- `ak.infomation`
- `ak.pretty_print_information`
- `ak.clear`
- `ak.disconnect`
- `ak.attach`
- `ak.GroupBy.attach`
- `ak.pdarray.unregister`
- `ak.Strings.unregister`
- `ak.Categorical.unregister`
- `ak.SegArray.unregister`
- `ak.GroupBy.unregister`
- `ak.shutdown`


#### Launching and Connecting to Arkounda Server
Be sure to follow the installation instructions on the [Arkouda README](https://github.com/mhmerrill/arkouda#readme) before running this notebook

In [1]:
import arkouda as ak

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2022.05.09+19.g81249870


The arkouda server must be running before connecting with `ak.connect`. The server can be launched by navigating to the arkouda directory and running `./arkouda_server -nl 1`. For more info, refer to [Running arkouda_server](https://github.com/mhmerrill/arkouda#running-arkouda_server-toc)

In [2]:
# connect to the arkouda server using the connect_url which the server prints out
ak.connect(connect_url="tcp://localhost:5555")

/home/josh/git/arkouda/arkouda/client.py:147: RuntimeWarning: Version mismatch between client (v2022.05.09+19.g81249870) and server (v2022.05.09+15.g8ea55dba.dirty); this may cause some commands to fail or behave incorrectly! Updating arkouda is strongly recommended.
  warnings.warn(('Version mismatch between client ({}) and server ({}); ' +
connected to arkouda server tcp://*:5555


#### Intializing and Registering Variables
We create random `pdarray`, `Strings`, `Categorical`, `SegArray`, and `GroupBy` objects and register them using in place `.register` functions

In [3]:
pda1 = ak.randint(0,10,100)
pda2 = ak.randint(0,10,100)
str1 = ak.random_strings_uniform(2, 5, 100)
str2 = ak.random_strings_uniform(2, 5, 100)


`Categorical` is built from `Strings` objects, so here we're using our `Strings` objects to create our `Categorical` objects

In [4]:
cat1 = ak.Categorical(str1)
cat2 = ak.Categorical(str2)

`SegArray` is a segmented array built from multiple `pdarray`s of varying length

In [5]:
# First, we generate three lists that are then flattened into a single list
a = [10, 11, 12, 13, 14, 15]
b = [20, 21]
c = [30, 31, 32, 33]
flat = a + b + c

# Then, we turn this flattened list into a pdarray
akflat = ak.array(flat)

# Next, we create a pdarray for the segments, which can be thought of as the starting index in the flattened array for each of the three intial lists
# In this case, 0 is the start of the first segment, list a; the length of list a, 6, is the start of the second segment, list b; and the combined length
# of lists a and b, 8, is the starting index of list c
segments = ak.array([0, len(a), len(a)+len(b)])

# Finally, we create our SegArry using the segments and akflat pdarrays
segarr = ak.SegArray(segments, akflat)

`GroupBy` is a grouping object that can group together multiple `groupable` objects (`pdarray`, `Strings`, and `Categorical`) or a list of these types. Each component can have multiple registered entries, so to keep things simple we will create two `GroupBy`s using a `pdarray` and `Strings` object

In [6]:
group1 = ak.GroupBy(ak.randint(0,10,100))
group2 = ak.GroupBy(str1)

In [7]:
pda1.register('pda1')
str1.register('str1')
cat1.register('cat1')
segarr.register('segarr')
group1.register('group1')

#### Verifying Registration and Using Information Methods
We have just registered `pdarray pda1`, `String str1`, `Categorical cat1`, `SegArray segarr`, and `GroupBy group1`. Note that `pdarray pda2`, `String str2`, `Categorical cat2`, and `GroupBy group2` have not been registered. We can verify this using `ak.list_registry`, `.is_registered`, or `.info`

In [8]:
# ak.list_registry returns a python list of all registered object names
print(ak.list_registry())

['segarr_segments', 'pda1', 'str1', 'segarr_lengths', 'cat1.codes', 'segarr_values', 'group1.permutation', 'group1.segments', 'group1_pdarray.keys', 'cat1.categories', 'cat1._akNAcode', 'cat1.segments', 'cat1.permutation', 'group1_pdarray.unique_keys']


Note that `SegArray` does not currently have a `.is_registered` method, so it is not included here

In [9]:
# Class level .is_registered() returns a boolean indicating the object's registration status
print(f'pda1 is registered: {pda1.is_registered()}')
print(f'pda2 is registered: {pda2.is_registered()}')
print(f'str1 is registered: {str1.is_registered()}')
print(f'str2 is registered: {str2.is_registered()}')
print(f'cat1 is registered: {cat1.is_registered()}')
print(f'cat2 is registered: {cat2.is_registered()}')
print(f'group1 is registered: {group1.is_registered()}')
print(f'group2 is registered: {group2.is_registered()}')

pda1 is registered: True
pda2 is registered: False
str1 is registered: True
str2 is registered: False
cat1 is registered: True
cat2 is registered: False
group1 is registered: True
group2 is registered: False


In [10]:
# Class level .info returns all attributes of an object in a JSON formatted string

print("str1.info():")
print(str1.info())

print("cat1.info():")
print(cat1.info())

str1.info():
[{"name":"str1", "dtype":"str", "size":100, "ndim":1, "shape":[100], "itemsize":1, "registered":true}]
cat1.info():
[{"name":"cat1.categories", "dtype":"str", "size":99, "ndim":1, "shape":[99], "itemsize":1, "registered":true},{"name":"cat1.segments", "dtype":"int64", "size":98, "ndim":1, "shape":[98], "itemsize":8, "registered":true},{"name":"cat1._akNAcode", "dtype":"int64", "size":1, "ndim":1, "shape":[1], "itemsize":8, "registered":true},{"name":"cat1.codes", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true},{"name":"cat1.permutation", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true}]


In [11]:
# Class level .pretty_print_info returns all the attributes from .info in human readable form

print("str1.pretty_print_info():")
str1.pretty_print_info()

print("cat1.pretty_print_info():")
print(cat1.pretty_print_info())

str1.pretty_print_info():
InfoEntry(name=str1, dtype=str, size=100, ndim=1, shape=[100], itemsize=1, registered=True)
cat1.pretty_print_info():
InfoEntry(name=cat1.categories, dtype=str, size=99, ndim=1, shape=[99], itemsize=1, registered=True)
InfoEntry(name=cat1.segments, dtype=int64, size=98, ndim=1, shape=[98], itemsize=8, registered=True)
InfoEntry(name=cat1._akNAcode, dtype=int64, size=1, ndim=1, shape=[1], itemsize=8, registered=True)
InfoEntry(name=cat1.codes, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=cat1.permutation, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
None


Using `ak.information` and `ak.pretty_print_information` it's easy to get attributes of all objects in the registry/symbol table

In [12]:
# ak.information returns all attributes of an object in a JSON formatted string
# ak.information can be called with a single object, all registered objects, or all objects

print("ak.information('registered_object_name'):")
print(ak.information('pda1'))

print('\nak.information(ak.RegisteredSymbols):')
print(ak.information(ak.RegisteredSymbols))

print('\nak.information(ak.AllSymbols):')
print(ak.information(ak.AllSymbols))

ak.information('registered_object_name'):
[{"name":"pda1", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true}]

ak.information(ak.RegisteredSymbols):
[{"name":"segarr_segments", "dtype":"int64", "size":3, "ndim":1, "shape":[3], "itemsize":8, "registered":true},{"name":"pda1", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true},{"name":"str1", "dtype":"str", "size":100, "ndim":1, "shape":[100], "itemsize":1, "registered":true},{"name":"segarr_lengths", "dtype":"int64", "size":3, "ndim":1, "shape":[3], "itemsize":8, "registered":true},{"name":"cat1.codes", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true},{"name":"segarr_values", "dtype":"int64", "size":12, "ndim":1, "shape":[12], "itemsize":8, "registered":true},{"name":"group1.permutation", "dtype":"int64", "size":100, "ndim":1, "shape":[100], "itemsize":8, "registered":true},{"name":"group1.segments", "dtype":"int64", "size":10,

In [13]:
# all the same arguments can be passed into ak.pretty_print_information for human readable output
print("ak.pretty_print_information('registered_object_name'):")
ak.pretty_print_information('pda1')

print('\nak.pretty_print_information(ak.RegisteredSymbols):')
ak.pretty_print_information(ak.RegisteredSymbols)

print('\nak.pretty_print_information(ak.AllSymbols):')
ak.pretty_print_information(ak.AllSymbols)

ak.pretty_print_information('registered_object_name'):
InfoEntry(name=pda1, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)

ak.pretty_print_information(ak.RegisteredSymbols):
InfoEntry(name=segarr_segments, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=pda1, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=str1, dtype=str, size=100, ndim=1, shape=[100], itemsize=1, registered=True)
InfoEntry(name=segarr_lengths, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=cat1.codes, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=segarr_values, dtype=int64, size=12, ndim=1, shape=[12], itemsize=8, registered=True)
InfoEntry(name=group1.permutation, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=group1.segments, dtype=int64, size=10, ndim=1, shape=[10], itemsize=8, registered=True)


We can see `ak.pretty_print_information(ak.RegisteredSymbols)` only contains references to objects related to `pda1` and `str1`

The JSON formmated string that's returned by `ak.information` and class level `.info` functions can be turned into a list of dictionaries in python using `json.loads` from the `json` library

In [14]:
# The JSON output of ak.information and .info can be parsed into python using the JSON library
import json

uint8_list = [symbol for symbol in json.loads(ak.information(ak.AllSymbols)) if symbol['dtype'] == 'uint8']
for sym in uint8_list:
    print(sym)

`ak.clear()` removes all non-registered objects from the symbol table, so `ak.pretty_print_information(ak.AllSymbols)` is different after a clear

In [15]:
print('Before clear:')
ak.pretty_print_information(ak.AllSymbols)

ak.clear()

print('\nAfter clear:')
ak.pretty_print_information(ak.AllSymbols)

Before clear:
InfoEntry(name=segarr_lengths, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=id_DH8F69g_46, dtype=int64, size=12, ndim=1, shape=[12], itemsize=8, registered=False)
InfoEntry(name=segarr_values, dtype=int64, size=12, ndim=1, shape=[12], itemsize=8, registered=True)
InfoEntry(name=id_DH8F69g_43, dtype=bool, size=3, ndim=1, shape=[3], itemsize=1, registered=False)
InfoEntry(name=cat1.permutation, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=segarr_segments, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=id_DH8F69g_48, dtype=int64, size=12, ndim=1, shape=[12], itemsize=8, registered=False)
InfoEntry(name=id_DH8F69g_49, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=False)
InfoEntry(name=id_DH8F69g_30, dtype=str, size=100, ndim=1, shape=[100], itemsize=1, registered=False)
InfoEntry(name=group1_pdarray.keys, dtype=int64, size=100, ndim=1, shape=

#### Disconnecting from Arkouda Server and Attaching Registered Objects
Users can `attach` to objects registered with the server after a disconnect. This enables access to registered objects even if the original python object is lost  

We are going to simulate the python client dying. To do this, we set references for `pda1` and `str1` to `None` and disconnect from the server

In [16]:
pda1 = None
str1 = None
cat1 = None
segarr = None
group1 = None

In [17]:
ak.disconnect()

disconnected from arkouda server tcp://*:5555


Now we come back and reconnect to the server. We attempt to access `pda1` and `str1` but no longer have the objects in python

In [18]:
# connect to the arkouda server using the connect_url which the server prints out
ak.connect("localhost")

connected to arkouda server tcp://*:5555


In [19]:
print(f'pda1:{pda1}')
print(f'str1:{str1}')
print(f'cat1:{cat1}')
print(f'segarr:{segarr}')
print(f'group1:{group1}')

pda1:None
str1:None
cat1:None
segarr:None
group1:None


However the server still has these objects registered

In [20]:
print('After reconnect to server:')
# ak.pretty_print_information and ak.information without arguments defaults to ak.RegisteredSymbols
ak.pretty_print_information()

After reconnect to server:
InfoEntry(name=segarr_segments, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=pda1, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=str1, dtype=str, size=100, ndim=1, shape=[100], itemsize=1, registered=True)
InfoEntry(name=segarr_lengths, dtype=int64, size=3, ndim=1, shape=[3], itemsize=8, registered=True)
InfoEntry(name=cat1.codes, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=segarr_values, dtype=int64, size=12, ndim=1, shape=[12], itemsize=8, registered=True)
InfoEntry(name=group1.permutation, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=group1.segments, dtype=int64, size=10, ndim=1, shape=[10], itemsize=8, registered=True)
InfoEntry(name=group1_pdarray.keys, dtype=int64, size=100, ndim=1, shape=[100], itemsize=8, registered=True)
InfoEntry(name=cat1.categories, dtype=str, size=99, ndim=1, shape

We want to attach to these objects on the server to regain access to them

In [21]:
pda1 = ak.util.attach('pda1')
str1 = ak.util.attach('str1')
cat1 = ak.util.attach('cat1', 'categorical')
segarr = ak.util.attach('segarr', 'segarray')
group1 = ak.GroupBy.attach('group1')


In [22]:
print(f'pda1:{pda1}')
print(f'str1:{str1}')
print(f'cat1:{cat1}')
print(f'segarr:{segarr}')
print(f'group1:{group1}')

pda1:[0 3 1 ... 2 4 8]
str1:['GNQE', 'CM', 'MBA', 'LVP', 'YX', 'JHHU', 'GIOQ', 'XJCM', 'NPNG', 'GZRW', 'JCBK', 'YVJZ', 'MMZ', 'QHR', 'SR', 'BU', 'IYLV', 'IC', 'WG', 'RO', 'MH', 'XBK', 'RCF', 'FUZ', 'KTMG', 'LXAI', 'FU', 'KNA', 'HZ', 'UUV', 'CNMG', 'EX', 'YA', 'QL', 'OQQJ', 'RX', 'RU', 'VMCU', 'FJM', 'YIJA', 'PMEL', 'ZZVQ', 'RT', 'YA', 'DPRM', 'BELD', 'EPJ', 'PTQ', 'MM', 'LSV', 'CPA', 'MUWH', 'VGB', 'JJDH', 'XD', 'NJZH', 'OGK', 'AWGM', 'ZL', 'BFA', 'JXV', 'SR', 'PA', 'TGF', 'JEE', 'CWH', 'NIV', 'PGEC', 'UEO', 'ZZND', 'MTWO', 'KON', 'UW', 'ID', 'JER', 'NVF', 'OP', 'THZF', 'EGSG', 'HR', 'NP', 'DFUU', 'CC', 'XQFZ', 'AK', 'XPR', 'AYCI', 'GOEO', 'LBWJ', 'MVGO', 'PAH', 'MSEA', 'CU', 'SVV', 'FW', 'MDXJ', 'BJZ', 'TGVJ', 'HKIG', 'VU']
cat1:['GNQE', 'CM', 'MBA', 'LVP', 'YX', 'JHHU', 'GIOQ', 'XJCM', 'NPNG', 'GZRW', 'JCBK', 'YVJZ', 'MMZ', 'QHR', 'SR', 'BU', 'IYLV', 'IC', 'WG', 'RO', 'MH', 'XBK', 'RCF', 'FUZ', 'KTMG', 'LXAI', 'FU', 'KNA', 'HZ', 'UUV', 'CNMG', 'EX', 'YA', 'QL', 'OQQJ', 'RX', 'RU', 'V

We restored our access to `pda1` and `str1` using the `attach` functionality. Now we unregister everything and shutdown the arkouda server 

In [23]:
pda1.unregister()
str1.unregister()
cat1.unregister()
segarr.unregister()
group1.unregister()
ak.clear()

In [24]:
print(ak.information(ak.AllSymbols))

[{"name":"cat1._akNAcode", "dtype":"int64", "size":1, "ndim":1, "shape":[1], "itemsize":8, "registered":true},{"name":"cat1.segments", "dtype":"int64", "size":98, "ndim":1, "shape":[98], "itemsize":8, "registered":true}]


In [25]:
ak.shutdown()